In [1]:
import calendar
import pandas as pd
import panel as pn

from datetime import date, timedelta
from sqlalchemy import create_engine

pn.extension('tabulator')

engine = create_engine("sqlite:///c:\\ruby\\port_lite\\db\\development.sqlite3")
conlite = engine.connect()

data_path = "../data/"
csv_path = "\\Users\\User\\iCloudDrive\\"
box_path = "\\Users\\User\\Dropbox\\"
one_path = "\\Users\\User\\OneDrive\\Documents\\Data\\"

today = date.today()
today

datetime.date(2022, 5, 23)

In [2]:
cols = 'trade name qty price active reason market xdate'.split()
colt = 'trade name qty target active reason market xdate'.split()

In [3]:
pd.read_sql_query('SELECT * FROM orders ORDER BY id DESC', conlite)

,id,trade,name,qty,price,active,reason,market,xdate
0,21,S,MAKRO,1500,40.50,0,C40.40,SET,2022-03-03
1,20,S,WHART,10000,12.00,0,C11.70,SET,2022-06-01
2,19,S,IP,2500,20.80,0,CP1S,mai,2022-02-02
3,18,S,DOHOME,2400,20.30,0,C20.25,SET100,2022-02-02
4,17,S,SYNEX,3000,28.25,0,CP1S,SET100,2022-03-10
5,16,S,NER,9000,7.50,0,CP1S,SET,2022-04-19
6,15,S,STA,2400,29.50,0,CUT,SET100,2022-04-19
7,14,S,JASIF,10000,12.00,0,C10.10,SET,2022-05-24
8,13,S,PTTGC,3000,58.00,0,CUT,SET50,2022-02-28
9,12,B,KKP,3000,68.00,0,3L,SET100,2022-04-28


### After call Ord-Log.bat which calculates spreads between target price and SET price.

In [4]:
file_name   = 'orders-log.csv'
input_file = data_path + file_name
#col_names   = ['trade','name','spreads','reason','market','qty','target_price','set_price','chg_amt','chg_pct','active']
df_log = pd.read_csv(input_file, sep=',',  index_col=None)
df_log

,trade,name,spd,reason,market,qty,target,current,change,percent,active,xdate
0,B,AP,-7,XXX,SET100,10000,10.00,10.70,-0.10,-0.93%,0,2022-05-10
1,B,ASK,-9,XXX,SET,3000,40.00,42.25,0.25,+0.60%,0,2022-03-03
2,B,BANPU,-1,XXX,SET50,12000,12.00,12.10,-0.10,-0.82%,0,2022-04-11
3,B,BBL,-9,1L,SET50,2400,124.00,128.50,2.00,+1.58%,0,2022-04-22
4,B,BCH,-4,RD10pct,SET100,3000,18.50,18.90,0.10,+0.53%,0,2022-05-10
5,B,BDMS,-4,1L,SET50,4200,26.00,27.00,0.00,0.00%,0,2022-03-09
6,B,BGC,-8,DOS,SET,10000,9.80,10.40,0.20,+1.96%,0,2022-05-25
7,B,DIF,2,DOS,SET,10000,14.20,14.00,0.00,0.00%,0,2022-05-17
8,B,HREIT,-5,RD05pct,SET,10000,8.20,8.45,0.00,0.00%,0,2022-05-25
9,B,IVL,-17,XXX,SET50,2400,45.00,49.25,0.50,+1.03%,0,2022-05-31


In [5]:
def calc(vals):
    spreads = vals
    if abs(spreads) < 10:
        return 1
    else:
        return 0

In [6]:
df_log['active'] = df_log['spd'].apply(calc)

In [7]:
df_log.groupby('active')['trade'].count()

active
0    10
1    11
Name: trade, dtype: int64

In [8]:
sql = '''
DELETE FROM orders'''
rp = conlite.execute(sql)
rp.rowcount

20

In [9]:
final = df_log[colt]
final

,trade,name,qty,target,active,reason,market,xdate
0,B,AP,10000,10.00,1,XXX,SET100,2022-05-10
1,B,ASK,3000,40.00,1,XXX,SET,2022-03-03
2,B,BANPU,12000,12.00,1,XXX,SET50,2022-04-11
3,B,BBL,2400,124.00,1,1L,SET50,2022-04-22
4,B,BCH,3000,18.50,1,RD10pct,SET100,2022-05-10
5,B,BDMS,4200,26.00,1,1L,SET50,2022-03-09
6,B,BGC,10000,9.80,1,DOS,SET,2022-05-25
7,B,DIF,10000,14.20,1,DOS,SET,2022-05-17
8,B,HREIT,10000,8.20,1,RD05pct,SET,2022-05-25
9,B,IVL,2400,45.00,0,XXX,SET50,2022-05-31


In [10]:
rcds = final.values.tolist()
rcds

[['B', 'AP', 10000, 10.0, 1, 'XXX', 'SET100', '2022-05-10'],
 ['B', 'ASK', 3000, 40.0, 1, 'XXX', 'SET', '2022-03-03'],
 ['B', 'BANPU', 12000, 12.0, 1, 'XXX', 'SET50', '2022-04-11'],
 ['B', 'BBL', 2400, 124.0, 1, '1L', 'SET50', '2022-04-22'],
 ['B', 'BCH', 3000, 18.5, 1, 'RD10pct', 'SET100', '2022-05-10'],
 ['B', 'BDMS', 4200, 26.0, 1, '1L', 'SET50', '2022-03-09'],
 ['B', 'BGC', 10000, 9.8, 1, 'DOS', 'SET', '2022-05-25'],
 ['B', 'DIF', 10000, 14.2, 1, 'DOS', 'SET', '2022-05-17'],
 ['B', 'HREIT', 10000, 8.2, 1, 'RD05pct', 'SET', '2022-05-25'],
 ['B', 'IVL', 2400, 45.0, 0, 'XXX', 'SET50', '2022-05-31'],
 ['B', 'KBANK', 1500, 140.0, 0, '5pct', 'SET50', '2022-04-18'],
 ['B', 'KKP', 3000, 68.0, 1, '3L', 'SET100', '2022-04-28'],
 ['S', 'DOHOME', 2400, 20.3, 0, 'C20.25', 'SET100', '2022-02-02'],
 ['S', 'IP', 2500, 20.3, 1, 'CP1S', 'mai', '2022-02-02'],
 ['S', 'JASIF', 10000, 12.0, 0, 'C10.10', 'SET', '2022-05-24'],
 ['S', 'MAKRO', 1500, 40.5, 0, 'C40.40', 'SET', '2022-03-03'],
 ['S', 'NER', 90

In [11]:
for rcd in rcds:
    print(rcd)

['B', 'AP', 10000, 10.0, 1, 'XXX', 'SET100', '2022-05-10']
['B', 'ASK', 3000, 40.0, 1, 'XXX', 'SET', '2022-03-03']
['B', 'BANPU', 12000, 12.0, 1, 'XXX', 'SET50', '2022-04-11']
['B', 'BBL', 2400, 124.0, 1, '1L', 'SET50', '2022-04-22']
['B', 'BCH', 3000, 18.5, 1, 'RD10pct', 'SET100', '2022-05-10']
['B', 'BDMS', 4200, 26.0, 1, '1L', 'SET50', '2022-03-09']
['B', 'BGC', 10000, 9.8, 1, 'DOS', 'SET', '2022-05-25']
['B', 'DIF', 10000, 14.2, 1, 'DOS', 'SET', '2022-05-17']
['B', 'HREIT', 10000, 8.2, 1, 'RD05pct', 'SET', '2022-05-25']
['B', 'IVL', 2400, 45.0, 0, 'XXX', 'SET50', '2022-05-31']
['B', 'KBANK', 1500, 140.0, 0, '5pct', 'SET50', '2022-04-18']
['B', 'KKP', 3000, 68.0, 1, '3L', 'SET100', '2022-04-28']
['S', 'DOHOME', 2400, 20.3, 0, 'C20.25', 'SET100', '2022-02-02']
['S', 'IP', 2500, 20.3, 1, 'CP1S', 'mai', '2022-02-02']
['S', 'JASIF', 10000, 12.0, 0, 'C10.10', 'SET', '2022-05-24']
['S', 'MAKRO', 1500, 40.5, 0, 'C40.40', 'SET', '2022-03-03']
['S', 'NER', 9000, 7.5, 0, 'CP1S', 'SET', '2022-

In [12]:
sql = """
INSERT INTO orders (trade, name, qty, price, active, reason, market, xdate)
VALUES (?, ?, ?, ?, ?, ?, ?, ?)
"""
print(sql)


INSERT INTO orders (trade, name, qty, price, active, reason, market, xdate)
VALUES (?, ?, ?, ?, ?, ?, ?, ?)



In [13]:
for rcd in rcds:
    conlite.execute(sql, rcd)

In [14]:
sql = '''
SELECT *
FROM orders
ORDER BY trade, name'''
df_orders = pd.read_sql(sql, conlite)
df_tab = pn.widgets.Tabulator(df_orders, layout='fit_data', width=800)
df_tab

Tabulator(layout='fit_data', value=    id trade  ..., width=800)

In [15]:
file_name = 'orders.csv'
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name
one_file = one_path + file_name

df_orders[cols].sort_values(['trade','name'],ascending=[True,True]).to_csv(output_file, header=True, index=False)
df_orders[cols].sort_values(['trade','name'],ascending=[True,True]).to_csv(data_file, header=True, index=False)
df_orders[cols].sort_values(['trade','name'],ascending=[True,True]).to_csv(box_file, header=True, index=False)
df_orders[cols].sort_values(['trade','name'],ascending=[True,True]).to_csv(one_file, header=True, index=False)